# Prever Sobreviventes no Titanic (classificação binária)

## Instalar Bibliotecas

In [0]:
%pip install xgboost databricks-feature-engineering

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/20/2f/2ccc91db71096cdb8c5174948db738075d7e38cfece0815500336f6e4ca8/xgboost-2.1.1-py3-none-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for databricks-feature-engineering from https://files.pythonhosted.org/packages/dd/62/7e4085fa5e550bec290cc1c5348ba1bd64cac58205ade06cdd4d8fb1dd43/databricks_feature_engineering-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for nvidia-nccl-cu12 from https://files.pythonhosted.org/packages/ed/1f/6482380ec8dcec4894e7503490fc536d846b0d59694acad9cf99f27d0e7d/nvidia_nccl_cu12-2.23.4-py3-none-manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for dbl-tempo<1,>=0.1.26 from https://files.pythonhosted.org/packages/61/0e/b7c4dc24221f24b39849a35424481dde08e3cafbbf95534312f256ad1632/dbl_temp

In [0]:
dbutils.library.restartPython()

## Analisar modelos

### Importar das bibliotecas


In [0]:
# Importando bibliotecas necessárias

# Para manipular os dados
import pandas as pd 

# Pré-processamento
from sklearn.impute import SimpleImputer  # Para imputação de valores ausentes
from sklearn.preprocessing import OneHotEncoder  # Para codificar variáveis categóricas
from sklearn.preprocessing import LabelEncoder # Para codificar variáveis categóricas
from sklearn.preprocessing import StandardScaler # Para padronizar os dados
from sklearn.preprocessing import MinMaxScaler # Para normalizar os dados

# Treinamento com Múltiplos Modelos de Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

# Seleção de Modelo
from sklearn.model_selection import train_test_split # Para dividir o dataset em treino e teste
from sklearn.model_selection import GridSearchCV # Mudar os argumentos dos modelos em busca de otimização(Hiperparameter tuning)

# Avaliação de resultados
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

# Para criar um pipeline
from sklearn.pipeline import Pipeline
x
# Biblioteca para acessar o Feature Store
from databricks.feature_store import FeatureStoreClient

# Para registrar o modelo
import mlflow
mlflow.set_registry_uri("databricks-uc")

import warnings # Para ignorar avisos
warnings.filterwarnings('ignore')


### (Extrair) Ingestão de dados

https://code.datasciencedojo.com/datasciencedojo/datasets

In [0]:
# Carregando o dataset Titanic
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
dados_titanic = pd.read_csv(url)

# Renomear as colunas para português
dados_titanic.rename(columns={
    'PassengerId': 'IdPassageiro',
    'Survived': 'Sobreviveu',
    'Pclass': 'Classe',
    'Name': 'Nome',
    'Sex': 'Sexo',
    'Age': 'Idade',
    'SibSp': 'IrmaosConjuges',
    'Parch': 'PaisFilhos',
    'Ticket': 'Bilhete',
    'Fare': 'Tarifa',
    'Cabin': 'Cabine',
    'Embarked': 'Embarque'
}, inplace=True)

# este comando só funciona no databricks. Se necessário, use head() para visualizar as primeiras linhas
display(dados_titanic)
# dados.head()

IdPassageiro,Sobreviveu,Classe,Nome,Sexo,Idade,IrmaosConjuges,PaisFilhos,Bilhete,Tarifa,Cabine,Embarque
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,null,S
6,0,3,"Moran, Mr. James",male,null,0,0,330877,8.4583,null,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,null,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,null,S
10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,null,C


### (Carga) Registro no Unity Catalog

In [0]:
# Registrar a tabela no Unity Catalog
dados_titanic_spark = spark.createDataFrame(dados_titanic)
dados_titanic_spark.write.format('delta').mode('overwrite').saveAsTable('coderhouse_ds_58690.default.tabela_titanic')

### Tratamento e Pré-processamento

In [0]:
# Verificar valores ausentes
dados_titanic.isnull().sum()

IdPassageiro        0
Sobreviveu          0
Classe              0
Nome                0
Sexo                0
Idade             177
IrmaosConjuges      0
PaisFilhos          0
Bilhete             0
Tarifa              0
Cabine            687
Embarque            2
dtype: int64

In [0]:
dados_titanic.head()

,IdPassageiro,Sobreviveu,Classe,Nome,Sexo,Idade,IrmaosConjuges,PaisFilhos,Bilhete,Tarifa,Cabine,Embarque
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
dados_titanic.Embarque.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [0]:
# Exemplo de uso do OneHotEncoder para transformar variáveis categóricas
encoder = OneHotEncoder(sparse=False)  # sparse=False retorna um array numpy
dados_categoricos = ['Sexo', 'Embarque']
dados_transformados = encoder.fit_transform(dados_titanic[dados_categoricos])

# Converter o array numpy de volta para um DataFrame para melhor visualização
colunas_transformadas = encoder.get_feature_names_out(dados_categoricos)
dados_transformados_df = pd.DataFrame(dados_transformados, columns=colunas_transformadas)

display(dados_transformados_df)

Sexo_female,Sexo_male,Embarque_C,Embarque_Q,Embarque_S,Embarque_nan
0.0,1.0,0.0,0.0,1.0,0.0
1.0,0.0,1.0,0.0,0.0,0.0
1.0,0.0,0.0,0.0,1.0,0.0
1.0,0.0,0.0,0.0,1.0,0.0
0.0,1.0,0.0,0.0,1.0,0.0
0.0,1.0,0.0,1.0,0.0,0.0
0.0,1.0,0.0,0.0,1.0,0.0
0.0,1.0,0.0,0.0,1.0,0.0
1.0,0.0,0.0,0.0,1.0,0.0
1.0,0.0,1.0,0.0,0.0,0.0


In [0]:
dados = dados_titanic.copy()
# NOVA LINHAS ENTRAM AQUI

# Remover colunas irrelevantes
dados.drop(['Nome', 'Bilhete', 'Cabine'], axis=1, inplace=True)

# Preencher valores ausentes em 'Idade' com a mediana
imputador = SimpleImputer(strategy='median') # Pode ser 'median', 'most_frequent', 'constant', etc.
dados['Idade'] = imputador.fit_transform(dados[['Idade']])

# Preencher valores ausentes em 'Embarque' com a moda
dados['Embarque'].fillna(dados['Embarque'].mode()[0], inplace=True)

# Feature Engineering
dados['TamanhoFamilia'] = dados['IrmaosConjuges'] + dados['PaisFilhos']
dados['EstaSozinho'] = (dados['TamanhoFamilia'] == 0).astype(int)

# Codificação numérica de variáveis categóricas
encoder = LabelEncoder()
dados['Sexo_Codificado'] = encoder.fit_transform(dados['Sexo'])
dados['Embarque_Codificado'] = encoder.fit_transform(dados['Embarque'])

# Remover colunas não mais necessárias
dados.drop(['Sexo', 'Embarque'], axis=1, inplace=True)

# Escalonar características numéricas (Padronizar e Normalizar)
caracteristicas = ['Idade', 'Tarifa', 'IrmaosConjuges', 'PaisFilhos', 'TamanhoFamilia']

# Padronizando os dados
scaler_standard = StandardScaler()
dados[caracteristicas] = scaler_standard.fit_transform(dados[caracteristicas])

# Normalizando os dados
scaler_minmax = MinMaxScaler()
dados[caracteristicas] = scaler_minmax.fit_transform(dados[caracteristicas])

dados_preprocessados = dados

# AQUI TERMINA A TRANSFORMAÇÃO
dados.head()

,IdPassageiro,Sobreviveu,Classe,Idade,IrmaosConjuges,PaisFilhos,Tarifa,TamanhoFamilia,EstaSozinho,Sexo_Codificado,Embarque_Codificado
0,1,0,3,0.271174,0.125,0.0,0.014151,0.1,0,1,2
1,2,1,1,0.472229,0.125,0.0,0.139136,0.1,0,0,0
2,3,1,3,0.321438,0.000,0.0,0.015469,0.0,1,0,2
3,4,1,1,0.434531,0.125,0.0,0.103644,0.1,0,0,2
4,5,0,3,0.434531,0.000,0.0,0.015713,0.0,1,1,2


In [0]:
# Verificar valores ausentes
dados.isnull().sum()

IdPassageiro      0
Sobreviveu        0
Classe            0
Sexo              0
Idade             0
IrmaosConjuges    0
PaisFilhos        0
Tarifa            0
Embarque          0
dtype: int64

In [0]:
# Realizar o pré-processamento e criar features
def preprocess_data(dados):
    # Copiar o DataFrame
    dados = dados_titanic.copy() 

    # Remover colunas irrelevantes
    dados.drop(['Nome', 'Bilhete', 'Cabine'], axis=1, inplace=True)

    # Preencher valores ausentes em 'Idade' com a mediana
    imputador = SimpleImputer(strategy='median') # Pode ser 'median', 'most_frequent', 'constant', etc.
    dados['Idade'] = imputador.fit_transform(dados[['Idade']])

    # Preencher valores ausentes em 'Embarque' com a moda
    dados['Embarque'].fillna(dados['Embarque'].mode()[0], inplace=True)

    # Feature Engineering
    dados['TamanhoFamilia'] = dados['IrmaosConjuges'] + dados['PaisFilhos']
    dados['EstaSozinho'] = (dados['TamanhoFamilia'] == 0).astype(int)

    # Codificação numérica de variáveis categóricas
    encoder = LabelEncoder()
    dados['Sexo_Codificado'] = encoder.fit_transform(dados['Sexo'])
    dados['Embarque_Codificado'] = encoder.fit_transform(dados['Embarque'])

    # Remover colunas não mais necessárias
    dados.drop(['Sexo', 'Embarque'], axis=1, inplace=True)

    # Escalonar características numéricas (Padronizar e Normalizar)
    caracteristicas = ['Idade', 'Tarifa', 'IrmaosConjuges', 'PaisFilhos', 'TamanhoFamilia']

    # Padronizando os dados
    scaler_standard = StandardScaler()
    dados[caracteristicas] = scaler_standard.fit_transform(dados[caracteristicas])

    # Normalizando os dados
    scaler_minmax = MinMaxScaler()
    dados[caracteristicas] = scaler_minmax.fit_transform(dados[caracteristicas])

    return dados

dados_preprocessados = preprocess_data(dados_titanic)

# Este comando só funciona no databricks.
# Se necessário, use head() para visualizar as primeiras linhas
display(dados_preprocessados)
# dados.head()

IdPassageiro,Sobreviveu,Classe,Idade,IrmaosConjuges,PaisFilhos,Tarifa,TamanhoFamilia,EstaSozinho,Sexo_Codificado,Embarque_Codificado
1,0,3,0.2711736617240512,0.125,0.0,0.014151057562208046,0.09999999999999999,0,1,2
2,1,1,0.47222920331741636,0.125,0.0,0.13913573538264068,0.09999999999999999,0,0,0
3,1,3,0.3214375471223925,0.0,0.0,0.015468569817999829,0.0,1,0,2
4,1,1,0.4345312892686604,0.125,0.0,0.10364429745562034,0.09999999999999999,0,0,2
5,0,3,0.4345312892686604,0.0,0.0,0.015712553569072384,0.0,1,1,2
6,0,3,0.34656948982156316,0.0,0.0,0.016509502093575767,0.0,1,1,1
7,0,1,0.6732847449107815,0.0,0.0,0.10122885832000206,0.0,1,1,2
8,0,3,0.01985423473234471,0.375,0.16666666666666663,0.04113566043083236,0.4,0,1,2
9,1,3,0.33400351847197784,0.0,0.3333333333333333,0.02173075436652839,0.19999999999999996,0,0,2
10,1,2,0.17064589092736862,0.125,0.0,0.058694292654020104,0.09999999999999999,0,0,0


In [0]:
# Verificar novamente valores ausentes
dados_preprocessados.isnull().sum()

IdPassageiro           0
Sobreviveu             0
Classe                 0
Idade                  0
IrmaosConjuges         0
PaisFilhos             0
Tarifa                 0
TamanhoFamilia         0
EstaSozinho            0
Sexo_Codificado        0
Embarque_Codificado    0
dtype: int64

### Criação de Features com o Feature Store

In [0]:
fs = FeatureStoreClient()

dados_preprocessados_spark = spark.createDataFrame(dados_preprocessados) # Transformar o pandas dataframe em um dataframe do Spark

# Criar uma tabela de features no Feature Store
fs.create_table(
    name='coderhouse_ds_58690.default.features_titanic',
    primary_keys='IdPassageiro',
    df=dados_preprocessados_spark,
    schema=dados_preprocessados_spark.schema,
    description='Dados de passageiros do Titanic',
    tags={'dataset': 'titanic', 'projeto': 'prod_gbs_finance_latam', 'estagio': 'teste', 'linguagem': 'python', 'framework': 'spark', 'linguagem_dados': 'pandas', 'data_carga': '2023-01-01', 'carga_automatica': 'nao', 'carga_manual': 'sim', 'tipo': 'feature_store', }
)

2024/10/02 00:05:01 INFO databricks.ml_features._compute_client._compute_client: Setting columns ['IdPassageiro'] of table 'coderhouse_ds_58690.default.features_titanic' to NOT NULL.
2024/10/02 00:05:04 INFO databricks.ml_features._compute_client._compute_client: Setting Primary Keys constraint ['IdPassageiro'] on table 'coderhouse_ds_58690.default.features_titanic'.
2024/10/02 00:05:15 WARNING databricks.ml_features._catalog_client._catalog_client_helper: Failed to record producer in the catalog. Notebook may have been renamed. Exception: {'error_code': 'RESOURCE_DOES_NOT_EXIST', 'message': "Notebook with path '/Shared/Modelos para Data Science /Machine Learning End to end' not found"}
2024/10/02 00:05:15 INFO databricks.ml_features._compute_client._compute_client: Created feature table 'coderhouse_ds_58690.default.features_titanic'.


<FeatureTable: name='coderhouse_ds_58690.default.features_titanic', table_id='bceedc8d-6085-409c-9aa1-33a2c290c26c', description='Dados de passageiros do Titanic', primary_keys=['IdPassageiro'], partition_columns=[], features=['IdPassageiro',
 'Sobreviveu',
 'Classe',
 'Idade',
 'IrmaosConjuges',
 'PaisFilhos',
 'Tarifa',
 'TamanhoFamilia',
 'EstaSozinho',
 'Sexo_Codificado',
 'Embarque_Codificado'], creation_timestamp=1727827500586, online_stores=[], notebook_producers=[], job_producers=[], table_data_sources=[], path_data_sources=[], custom_data_sources=[], timestamp_keys=[], tags={'carga_automatica': 'nao',
 'carga_manual': 'sim',
 'data_carga': '2023-01-01',
 'dataset': 'titanic',
 'estagio': 'teste',
 'framework': 'spark',
 'linguagem': 'python',
 'linguagem_dados': 'pandas',
 'projeto': 'prod_gbs_finance_latam',
 'tipo': 'feature_store'}>

### Divisão dos Dados (Data Splitting)

Dividimos os dados em dois grupos:

- **Dados de Treinamento:** Usados para ensinar o modelo a identificar padrões.  
- **Dados de Teste:** Usados para verificar se o modelo aprendeu bem e faz previsões corretas em situações novas.  

Essa divisão é importante para garantir que o modelo não apenas memorize os exemplos, mas realmente aprenda a generalizar para dados que nunca viu antes. Assim, asseguramos que ele funcionará bem no mundo real.

In [0]:
dados_preprocessados.head()

,IdPassageiro,Sobreviveu,Classe,Idade,IrmaosConjuges,PaisFilhos,Tarifa,TamanhoFamilia,EstaSozinho,Sexo_Codificado,Embarque_Codificado
0,1,0,3,0.271174,0.125,0.0,0.014151,0.1,0,1,2
1,2,1,1,0.472229,0.125,0.0,0.139136,0.1,0,0,0
2,3,1,3,0.321438,0.000,0.0,0.015469,0.0,1,0,2
3,4,1,1,0.434531,0.125,0.0,0.103644,0.1,0,0,2
4,5,0,3,0.434531,0.000,0.0,0.015713,0.0,1,1,2


In [0]:
X = dados_preprocessados.drop(['IdPassageiro', 'Sobreviveu'], axis=1)
X.head()

,Classe,Idade,IrmaosConjuges,PaisFilhos,Tarifa,TamanhoFamilia,EstaSozinho,Sexo_Codificado,Embarque_Codificado
0,3,0.271174,0.125,0.0,0.014151,0.1,0,1,2
1,1,0.472229,0.125,0.0,0.139136,0.1,0,0,0
2,3,0.321438,0.000,0.0,0.015469,0.0,1,0,2
3,1,0.434531,0.125,0.0,0.103644,0.1,0,0,2
4,3,0.434531,0.000,0.0,0.015713,0.0,1,1,2


In [0]:
y = dados_preprocessados['Sobreviveu']
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Sobreviveu, dtype: int64

In [0]:
# Separar features (caracteristicas) e target (alvo)

# Dividir em conjuntos de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [0]:
dados_preprocessados.shape

(891, 11)

In [0]:
X_treino.shape

(712, 9)

In [0]:
X_teste.shape

(179, 9)

### Teste de Vários Modelos

Lista de Modelos a Serem Testados. 

### **Modelos de classificação binária**
- Regressão Logística
- Árvore de Decisão
- Random Forest
- KNN
- MLP
- Máquina de Vetores de Suporte (SVM)
- Gradient Boosting
- XGBoost

In [0]:
# Dicionário para armazenar os resultados
modelos = {
    'Regressão Logística': LogisticRegression(),
    'Árvore de Decisão': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(probability=True),
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [0]:
modelos

{'Regressão Logística': LogisticRegression(),
 'Árvore de Decisão': DecisionTreeClassifier(),
 'KNN': KNeighborsClassifier(),
 'MLP': MLPClassifier(max_iter=1000),
 'Random Forest': RandomForestClassifier(),
 'SVM': SVC(probability=True),
 'Gradient Boosting': GradientBoostingClassifier(),
 'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='logloss',
               feature_types=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=None, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=None,


In [0]:
# Lista para armazenar os resultados
resultados = []

# Avaliação de cada modelo
for nome_modelo, modelo in modelos.items():
    # Treinar o modelo
    modelo.fit(X_treino, y_treino)  # << aqui é executado o treinamento do modelo 

    # Previsões
    y_pred = modelo.predict(X_teste) # << aqui é feito a previsão diante de um modelo treinado

    # Acurácia
    acuracia = accuracy_score(y_teste, y_pred) # <<  comparar o que foi previsto com os dados que nos já sabemos que sao respostas corretas
    auc  = roc_auc_score(y_teste, modelo.predict_proba(X_teste)[:, 1]) # Prever as probabilidades no conjunto de teste

    # Adicionar os resultados a lista
    resultados.append({
        "Modelo": nome_modelo,
        "Acurácia": acuracia,
        "AUC-ROC": auc 
    })
    print(f'{nome_modelo}  - Acurácia: {acuracia:.2f}, AUC-ROC: {auc:.2f}')

Regressão Logística  - Acurácia: 0.80, AUC-ROC: 0.85
Árvore de Decisão  - Acurácia: 0.80, AUC-ROC: 0.78
KNN  - Acurácia: 0.79, AUC-ROC: 0.83
MLP  - Acurácia: 0.82, AUC-ROC: 0.85
Random Forest  - Acurácia: 0.80, AUC-ROC: 0.84
SVM  - Acurácia: 0.82, AUC-ROC: 0.81
Gradient Boosting  - Acurácia: 0.80, AUC-ROC: 0.82
XGBoost  - Acurácia: 0.80, AUC-ROC: 0.81


**Cálculo Simplificado do Accuracy Score (Acurácia)**

O `accuracy_score` mede a proporção de previsões corretas feitas pelo modelo em relação ao total de previsões. É calculado assim:

- **Acurácia = (Número de Previsões Corretas) ÷ (Número Total de Previsões)**

**Exemplo:**

Se o modelo fez 100 previsões e acertou 85 delas:
- **Acurácia = 85 ÷ 100 = 0,85 ou 85%**

**Resumo:**
- O `accuracy_score` indica a porcentagem de acertos do modelo.
- Quanto mais próximo de 100%, melhor o desempenho em prever corretamente.

In [0]:
# Criar DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)
df_resultados['Combinada'] = df_resultados['Acurácia'] * df_resultados['AUC-ROC']

# Identificar o melhor modelo com base na métrica AUC
for valor in [ 'Acurácia', 'AUC-ROC', 'Combinada']:
    melhor_modelo_nome = df_resultados.sort_values(by=valor, ascending=False).iloc[0]['Modelo']
    print(f"Melhor modelo {valor}: {melhor_modelo_nome}")

df_resultados.sort_values(by='Acurácia', ascending=False)

Melhor modelo Acurácia: MLP
Melhor modelo AUC-ROC: Regressão Logística
Melhor modelo Combinada: MLP


,Modelo,Acurácia,AUC-ROC,Combinada
3,MLP,0.821229,0.853228,0.700696
5,SVM,0.815642,0.813834,0.663798
0,Regressão Logística,0.804469,0.854414,0.687350
4,Random Forest,0.804469,0.835968,0.672511
6,Gradient Boosting,0.804469,0.819829,0.659527
1,Árvore de Decisão,0.798883,0.782609,0.625213
7,XGBoost,0.798883,0.812385,0.649000
2,KNN,0.787709,0.832806,0.656009


In [0]:
# Identificar o melhor modelo com base na métrica AUC
melhor_modelo_nome = df_resultados.sort_values(by="Combinada", ascending=False).iloc[0]['Modelo']
print(f"Melhor modelo: {melhor_modelo_nome}")

Melhor modelo: MLP


### Otimização de Hiperparâmetros 
(exemplo com Regressão Logística)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Definir o modelo de Regressão Logística
regressao_logistica = LogisticRegression()

# Proposta de hiperparâmetros
param_grid = {
    'modelo__penalty': ['l1', 'l2', 'elasticnet', 'none'],  # Penalização (regularização)
    'modelo__C': [0.001, 0.01, 0.1, 1, 10, 100],            # Força de regularização (inverso da regularização)
    'modelo__solver': ['liblinear', 'saga', 'lbfgs'],       # Solver (otimizador)
    'modelo__fit_intercept': [True, False],                 # Incluir ou não o intercepto
    'modelo__max_iter': [100, 200, 500]                     # Número máximo de iterações
}

# Definir o pipeline com o pré-processador e o modelo
pipeline = Pipeline(steps=[('modelo', regressao_logistica)])

# Otimização de hiperparâmetros com GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Treinar o modelo com a busca em grade
grid_search.fit(X_treino, y_treino)

# Exibir os melhores hiperparâmetros e o melhor desempenho
print(f'Melhores parâmetros: {grid_search.best_params_}')
print(f'Melhor acurácia: {grid_search.best_score_:.4f}')

# cria modelo com os parâmetros mencionados
modelo = LogisticRegression(penalty='l1', C=0.001, solver='liblinear', fit_intercept=True, max_iter=100)

### Análise do Melhor Modelo

In [0]:
# Avaliação detalhada do Estimador (Modelo) de aprendizagem automática
modelo = LogisticRegression()

# Treinar o modelo
modelo.fit(X_treino, y_treino)

# Previsões
y_pred = modelo.predict(X_teste)

# Acurácia
acuracia = accuracy_score(y_teste, y_pred)
auc  = roc_auc_score(y_teste, modelo.predict_proba(X_teste)[:, 1]) # Prever as probabilidades no conjunto de teste

print(f'Regressão Logística  - Acurácia: {acuracia:.2f}, AUC-ROC: {auc:.2f}')

**Matriz de Confusão**

A matriz de confusão é uma tabela que resume o desempenho de um modelo de classificação, comparando as previsões feitas pelo modelo com os valores reais. Ela mostra:

**- Verdadeiros Positivos (VP):** Quantidade de vezes que o modelo previu corretamente a classe positiva.  
**- Falsos Positivos (FP):** Quando o modelo previu a classe positiva, mas o valor real era negativo.  
**- Falsos Negativos (FN):** Quando o modelo previu a classe negativa, mas o valor real era positivo.  
**- Verdadeiros Negativos (VN):** Quantidade de vezes que o modelo previu corretamente a classe negativa.

A matriz permite identificar onde o modelo está acertando e errando, ajudando a entender melhor seu desempenho e a ajustar melhorias.

In [0]:
print('Matriz de Confusão:')
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_teste, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predições')
plt.ylabel('Real')
plt.title('Matriz Confusão - Confusion Matrix')
plt.show()

In [0]:
print('Relatório de Classificação:')
report = classification_report(y_teste, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report.head()

O `classification_report` é uma função do scikit-learn que fornece uma avaliação detalhada do desempenho de um modelo de classificação. Ele apresenta métricas-chave para cada classe, incluindo:

**- Precisão (Precision):** Proporção de previsões positivas corretas em relação ao total de previsões positivas feitas para uma classe.  
**- Revocação (Recall):** Proporção de verdadeiros positivos identificados corretamente em relação ao total de reais positivos da classe.  
**- F1-Score:** Média harmônica entre precisão e revocação, balanceando ambas as métricas.   
**- Suporte (Support):** Número de ocorrências reais de cada classe no conjunto de teste.  

O relatório ajuda a entender como o modelo está performando em cada classe individualmente, destacando possíveis desequilíbrios ou áreas que necessitam de melhorias.

## Implementação no Azure Databricks

### Experiment Tracking com MLflow

In [0]:
import mlflow
import mlflow.sklearn
from databricks.feature_store import FeatureStoreClient, FeatureLookup
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Definir o caminho correto do experimento
parent_folder = "/Users/jefferson.klister@hydro.com/Machine Learning"
nome_experimento = "experimento_titanic"
caminho_experimento = f"{parent_folder}/{nome_experimento}"
artifact_path_name = 'modelo_regressao_logistica'

# Verificar se o experimento já existe
experimento = mlflow.get_experiment_by_name(caminho_experimento)

if experimento:
    # Se o experimento existir, definir o contexto do experimento usando seu ID
    mlflow.set_experiment(experiment_id=experimento.experiment_id)
else:
    # Se o experimento não existir, criar um novo e definir o contexto
    experiment_id = mlflow.create_experiment(name=caminho_experimento)
    mlflow.set_experiment(experiment_id=experiment_id)

# Instanciar o FeatureStoreClient
fs = FeatureStoreClient()

with mlflow.start_run(run_name=melhor_modelo_nome):
    feature_names = [
        'Classe', 'Idade', 'Tarifa', 'IrmaosConjuges', 'PaisFilhos',
        'Tarifa', 'TamanhoFamilia', 'EstaSozinho', 'Sexo_Codificado', 'Embarque_Codificado'
    ]

    # Remover duplicatas
    feature_names = list(set(feature_names))

    # Carregar dados do Feature Store
    feature_lookups = [
        FeatureLookup(
            table_name='prod_gbs_finance_latam.sandbox.features_titanic',
            feature_names=feature_names,
            lookup_key='IdPassageiro'
        )
    ]
    
    # Ler a tabela do Unity Catalog
    df_unity_catalog = spark.read.table('prod_gbs_finance_latam.sandbox.features_titanic')

    # Preparar os dados de treinamento
    training_set = fs.create_training_set(
        df_unity_catalog.select('IdPassageiro', 'Sobreviveu'),
        feature_lookups=feature_lookups,
        label='Sobreviveu',
        exclude_columns=['IdPassageiro']
    )
    treinamento_df = training_set.load_df().toPandas()

    # Separar features e target
    X = treinamento_df.drop('Sobreviveu', axis=1)
    y = treinamento_df['Sobreviveu']

    # Dividir os dados em conjuntos de treinamento e teste
    X_treino, X_teste, y_treino, y_teste = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Treinar o modelo
    modelo = LogisticRegression()
    modelo.fit(X_treino, y_treino)

    # Previsões
    y_pred = modelo.predict(X_teste)

    # Acurácia
    acuracia = accuracy_score(y_teste, y_pred)
    auc_roc = roc_auc_score(y_teste, modelo.predict_proba(X_teste)[:, 1])

    print(f'Regressão Logística - Acurácia: {acuracia:.2f}, AUC-ROC: {auc_roc:.2f}')
        
    # Salvando métricas no MLflow
    mlflow.log_metric('acuracia', acuracia)
    mlflow.log_param('modelo', 'Regressão Logística')
    mlflow.log_metric('auc_roc', auc_roc)
    
    # Logar hiperparâmetros do modelo
    hiperparametros = modelo.get_params()
    mlflow.log_params(hiperparametros)

    # Registrar o modelo no MLflow usando o Feature Store
    fs.log_model(
        model=modelo,
        artifact_path=artifact_path_name,
        flavor=mlflow.sklearn,
        training_set=training_set
    )

    run_id = mlflow.active_run().info.run_id
    model_uri = f"runs:/{run_id}/modelo_regressao_logistica"
    model_details = mlflow.register_model(
        model_uri=model_uri,
        name='prod_gbs_finance_latam.sandbox.titanic_regressao_logistica_modelo'
    )


### Registro do Modelo no Model Registry

In [0]:
# Registrar o modelo
from mlflow.tracking import MlflowClient

client = MlflowClient()
experimento = mlflow.get_experiment_by_name(caminho_experimento)
runs = client.search_runs(experiment_ids=[experimento.experiment_id], filter_string=f"tags.mlflow.runName = '{melhor_modelo_nome}'", order_by=["metrics.AUC DESC"], max_results=1)
best_run = runs[0]

# Registrar o modelo no Unity Catalog
model_name = "prod_gbs_finance_latam.sandbox.titanic_regressao_logistica_modelo"
model_uri = f"runs:/{best_run.info.run_id}/{artifact_path_name}"

# Registrar o modelo
model_details = mlflow.register_model(model_uri=model_uri, name=model_name)

###Transição do estágio do modelo

In [0]:
client = MlflowClient()

# Transição do estágio do modelo
# Definir o alias 'Production' para a versão atual do modelo
client = mlflow.tracking.MlflowClient()
client.set_registered_model_alias(
    name='prod_gbs_finance_latam.sandbox.titanic_regressao_logistica_modelo',
    alias='Production',
    version=model_details.version
)

In [0]:
# Opcional: Remover o alias 'Production' de versões anteriores
# Obter todas as versões do modelo
model_versions = client.get_model_version_download_uri(
    name='prod_gbs_finance_latam.sandbox.titanic_regressao_logistica_modelo',
    version=model_details.version
)

In [0]:
# Carregar o modelo usando o alias 'Production'
modelo_producao = mlflow.pyfunc.load_model(
    model_uri='models:/prod_gbs_finance_latam.sandbox.titanic_regressao_logistica_modelo@Production'
)


### Implantação e Serving do Modelo via API

In [0]:
# import requests
# import json

# # Endpoint do modelo
# url = 'https://<sua-instancia-databricks>/model/titanic_random_forest_modelo/Production/invocations'

# # Dados de entrada no formato JSON
# # Exemplo com um único registro
# entrada = {
#     'dataframe_records': [
#         {
#             'Classe': 3,
#             'Sexo': 1,
#             'Idade': 22.0,
#             'Tarifa': 7.25,
#             'IrmaosConjuges': 1,
#             'PaisFilhos': 0,
#             'Embarque': 1
#         }
#     ]
# }

# # Cabeçalhos com autenticação
# headers = {
#     'Authorization': f'Bearer {DATABRICKS_TOKEN}',
#     'Content-Type': 'application/json'
# }

# # Realizar a requisição
# response = requests.post(url, headers=headers, data=json.dumps(entrada))

# # Obter as previsões
# predicoes = response.json()
# print(predicoes)
